In [ ]:
from utils import *
processed_folder = get_processed_folder()
routes = load_routes()

# Daily station health
# Save to vds_health_{cfg_name}.csv

In [ ]:
for route_name, route_vdss in routes.items():

    print(route_name)

    # Collect all days from hourly files for these vdss
    days_set = set()
    for vds in route_vdss:

        try:
            filename = os.path.join(processed_folder,f'{vds}_hourly.csv')
            df_vds = pd.read_csv(filename,index_col='timestamp')
            days = set(pd.Timestamp(t).date() for t in df_vds.index)
            days_set.update(days)
        except:
            print(f"Warning: {vds}_hourly.csv not found.")

    route_days = pd.Series(list(days_set))
    route_days = route_days.sort_values()

    # vds_health
    vds_health = pd.DataFrame(index=route_days, columns=route_vdss)

    for vds in route_vdss:

        try:
            filename = os.path.join(processed_folder,f'{vds}_hourly.csv')
            df_vds = pd.read_csv(filename)
            df_vds['date'] = [pd.Timestamp(t).date() for t in df_vds['timestamp']]

            for day in route_days:
                ind = df_vds['date']==day
                vds_health.loc[day,vds] = df_vds.loc[ind,'perc_obs'].mean(skipna=True)

        except:
            print(f"Warning: {vds}_hourly.csv not found.")

    # Save to file
    filename = os.path.join(processed_folder,f'vds_health_{route_name}.csv')
    vds_health.to_csv(filename)